In [ ]:
import cv2
import numpy as np
from copy import deepcopy

# Initialize the camera
camera = cv2.VideoCapture(0)

# Capture an image from the camera
return_value, image = camera.read()

# Save the captured image
cv2.imwrite('image.jpeg', image)

# Release the camera
camera.release()

# Process the image
frame = image
edge_img = deepcopy(image)

# Perform edge detection
edged = cv2.Canny(edge_img, 50, 100)
edged = cv2.dilate(edged, None, iterations=1)
edged = cv2.erode(edged, None, iterations=1)

# Find contours
cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
if cnts:
    max_cont = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_cont)
else:
    print("No contours found")
    x = y = w = h = 0

# Convert the image to HSV color space
hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# Create masks for different shades of red
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
redmask1 = cv2.inRange(hsv, lower_red1, upper_red1)

lower_red2 = np.array([170, 100, 100])
upper_red2 = np.array([180, 255, 255])
redmask2 = cv2.inRange(hsv, lower_red2, upper_red2)

# Combine the red masks
redmask = redmask1 + redmask2

# Define kernels for morphological operations
kernelOpen = np.ones((5, 5), np.uint8)
kernelClose = np.ones((10, 10), np.uint8)

# Apply morphological operations
maskOpen = cv2.morphologyEx(redmask, cv2.MORPH_OPEN, kernelOpen)
maskClose = cv2.morphologyEx(maskOpen, cv2.MORPH_CLOSE, kernelClose)
maskFinal = maskClose

# Display the final red mask
cv2.imshow('Red Mask', maskFinal)

# Find contours in the red mask
cnt_r, _ = cv2.findContours(maskFinal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnt_r_area = sum(cv2.contourArea(c) for c in cnt_r)
# print(f"Redness Area: {cnt_r_area}")

# Create masks for other colors
lower_green = np.array([40, 40, 40])
upper_green = np.array([80, 255, 255])
greenmask = cv2.inRange(hsv, lower_green, upper_green)
cv2.imshow("Green Mask", greenmask)
cnt_g = cv2.countNonZero(greenmask)
# print("Greenness:", cnt_g)

lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])
yellowmask = cv2.inRange(hsv, lower_yellow, upper_yellow)
cv2.imshow('Yellow Mask', yellowmask)
cnt_y = cv2.countNonZero(yellowmask)
# print("Yellowness:", cnt_y)

# Calculate the total area and percentages
tot_area = cnt_r_area + cnt_y + cnt_g
if tot_area == 0:
    print("No significant area detected")
else:
    rperc = cnt_r_area / tot_area
    yperc = cnt_y / tot_area
    gperc = cnt_g / tot_area
    glimit = 0.1

    # Debug prints to understand the areas
    # print(f"Total Area: {tot_area}")
    print(f"Redness Percentage: {rperc * 100:.2f}%")
    print(f"Greenness Percentage: {gperc * 100:.2f}%")
    print(f"Yellowness Percentage: {yperc * 100:.2f}%")

    if rperc * 100 < 10 or yperc * 100 < 10 or gperc * 100 < 10:
        if gperc > glimit:
            print(f"Unripe ({gperc * 100:.2f}% Green)")
        else:
            print(f"Ripe ({rperc * 100:.2f}% Red)")
    else:
        print("Fruit not detected")
# Wait for a key press and close all windows
cv2.waitKey(0)
cv2.destroyAllWindows()